Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [53]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [54]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 999)
# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [55]:
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space'],
      dtype='object')

In [56]:
df.created = pd.to_datetime(df.created)

In [57]:
df['building_benefits'] = df[['fitness_center', 'high_speed_internet', 'elevator', 'swimming_pool', 'common_outdoor_space', 'laundry_in_building', 'roof_deck', 'wheelchair_access']].apply(sum, axis=1)

In [58]:
df['cool_rooms'] = df[['bathrooms', 'bedrooms', 'dining_room',
                       'terrace', 'loft', 'garden_patio',
                       'outdoor_space', 'balcony']].apply(sum, axis=1)

In [59]:
df.building_benefits.value_counts(), df.cool_rooms.value_counts()

(0    20760
 1    13559
 2     6189
 3     3708
 4     3141
 5     1064
 6      377
 7       14
 8        5
 Name: building_benefits, dtype: int64, 2.0     14476
 3.0     10659
 4.0      7170
 1.0      6766
 5.0      3784
 6.0      2280
 7.0      1271
 8.0       612
 9.0       340
 10.0      208
 4.5       203
 3.5       173
 5.5       171
 0.0       142
 2.5       112
 6.5       109
 11.0       95
 7.5        83
 8.5        47
 12.0       38
 10.5       24
 9.5        21
 12.5        9
 13.0        6
 1.5         6
 11.5        5
 16.0        4
 15.0        2
 13.5        1
 Name: cool_rooms, dtype: int64)

In [60]:
train = df[df.created.apply(getattr, args=['month']) < 6]
test  = df[df.created.apply(getattr, args=['month']) > 5]

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
model = LinearRegression()

In [63]:
features = ['building_benefits', 'cool_rooms']
target = 'price'
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [64]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [65]:
y_pred = model.predict(X_test)
y_pred

array([4198.21284373, 3568.31049025, 2774.05587018, ..., 2774.05587018,
       3343.7186596 , 3456.01457492])

In [66]:
model.coef_

array([112.29591532, 569.66278942])

In [67]:
model.intercept_

1634.7302913292515

In [68]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [70]:
mean_squared_error(y_test, y_pred)

1862519.256353486

In [69]:
mean_absolute_error(y_test, y_pred)

906.3228688885696

In [71]:
r2_score(y_test, y_pred)

0.4007379307155243